# コード8-1：NLMEMを用いたパラメータ推定の実装

『ウイルス感染の数理モデルとシミュレーション ーデータを定量的に理解するー』

NLMEMを用いたパラメータ推定の実装の例をまとめる。

## Monolixを用いた非線形混合効果モデルの実装

一般的な非線形混合効果モデルを実装するためには複数のパッケージ等が存在するが、それらを用いてパラメータ推定のアルゴリズムであるstochastic approximation expectation maximization（SAEM）とODEの数値計算を高速に実行するためには、工夫が必要であると考えられる。Monolixではこれらの点が解決されるインターフェースが準備されているので使い勝手が良い。

GUIでも利用することができるが、ここではR APIを用いた実装を紹介する。

学術利用（非商用）で無料のライセンスの取得が必須である。

また、2024年2月現在、MonolixSuite2023R1においてはApple Silicon用のアプリがリリースされていないことから、Rosettaによる変換が必要であり、R APIを利用することはできない。


- Monolix  
https://lixoft.com/products/monolix/
- Monolix Documentation  
https://monolix.lixoft.com/
- R API  
https://monolix.lixoft.com/monolix-api/

In [1]:
## initialization
## https://monolix.lixoft.com/monolix-api/lixoftconnectors_installation/
#install.packages('RJSONIO')
#install.packages("~/Lixoft/MonolixSuite2023R1/connectors/lixoftConnectors.tar.gz", repos = NULL, type="source", INSTALL_opts ="--no-multiarch")

## データ

In [2]:
dataPath <- "../../data/08_01/data_08_01.csv"
head(read.csv(dataPath))

,time,VL,id,cens,limit
,<int>,<dbl>,<int>,<chr>,<dbl>
1,19,100.0000,1,1,1e-04
2,13,100.0000,1,1,1e-04
3,10,219.9138,1,.,1e-04
4,15,100.0000,1,1,1e-04
5,2,348243.3116,1,.,1e-04
6,1,193073.6141,1,.,1e-04


In [26]:
min(read.csv(dataPath)$VL)

[1] 100

In [4]:
dataTypes <- c("time", "observation", "id", "cens", "limit")
observationTypes <- list("VL" = "continuous")

## モデル

In [5]:
modelPath <- "../../data/08_01/model_08_01.txt"

## 新規プロジェクト or プロジェクト読み込み

In [6]:
library(lixoftConnectors)
initializeLixoftConnectors("monolix")

Loading required package: RJSONIO

[INFO] The directory specified in the initialization file of the Lixoft Suite (located at "/home/iwanami/lixoft/lixoft.ini") will be used by default: "/home/iwanami/Lixoft/MonolixSuite2023R1"

[INFO] The lixoftConnectors package has been successfully initialized:
lixoftConnectors package version -> 2023.1
Lixoft softwares suite version   -> 2023R1



In [7]:
name_project_file <- "outputs/08_01/virusdynamics.mlxtran"

In [8]:
mappingList <- list(list(data = "VL", prediction = "VL", model = "y1"))

newProject(data = list(dataFile = dataPath,
                       headerTypes = dataTypes,
                       observationTypes = observationTypes,
                       mapping = mappingList),
           modelFile = modelPath)

saveProject(projectFile = name_project_file)

In [9]:
if (!isProjectLoaded()){
    loadProject(name_project_file)
}

## 観測モデルと誤差モデルの設定

In [10]:
getContinuousObservationModel()

$prediction
 VL_ 
"VL" 

$formula
                          VL_ 
"VL_ = VL + (a + b*VL) * e\n" 

$distribution
     VL_ 
"normal" 

$errorModel
        VL_ 
"combined1" 

$parameters
$parameters$VL_
[1] "a" "b" "c"


$autocorrelation
  VL_ 
FALSE

In [11]:
setMapping(list(list(data = "VL", prediction = "VL", model = "y1")))

In [12]:
setObservationDistribution(y1 = "logNormal")
setErrorModel(y1 = "constant")

In [13]:
getContinuousObservationModel()

$prediction
  y1 
"VL" 

$formula
                           y1 
"log(y1) = log(VL) + a * e\n" 

$distribution
         y1 
"logNormal" 

$errorModel
        y1 
"constant" 

$parameters
$parameters$y1
[1] "a"


$autocorrelation
   y1 
FALSE

### パラメータの分布と混合効果の設定

In [14]:
getIndividualParameterModel()

$name
[1] "beta"  "gamma" "delta" "V0"   

$distribution
       beta       gamma       delta          V0 
"logNormal" "logNormal" "logNormal" "logNormal" 

$limits
named list()

$formula
[1] "log(beta) = log(beta_pop) + eta_beta\nlog(gamma) = log(gamma_pop) + eta_gamma\nlog(delta) = log(delta_pop) + eta_delta\nlog(V0) = log(V0_pop) + eta_V0\n"

$variability
$variability$id
 beta gamma delta    V0 
 TRUE  TRUE  TRUE  TRUE 


$covariateModel
$covariateModel$beta
named list()

$covariateModel$gamma
named list()

$covariateModel$delta
named list()

$covariateModel$V0
named list()

In [15]:
indivParamModel <- list(variability = list(id = c(beta = TRUE,
                                                  gamma = TRUE,
                                                  delta = TRUE,
                                                  V0 = TRUE)),
                        covariateModel = list())



setIndividualParameterModel(indivParamModel)

In [16]:
getIndividualParameterModel()

$name
[1] "beta"  "gamma" "delta" "V0"   

$distribution
       beta       gamma       delta          V0 
"logNormal" "logNormal" "logNormal" "logNormal" 

$limits
named list()

$formula
[1] "log(beta) = log(beta_pop) + eta_beta\nlog(gamma) = log(gamma_pop) + eta_gamma\nlog(delta) = log(delta_pop) + eta_delta\nlog(V0) = log(V0_pop) + eta_V0\n"

$variability
$variability$id
 beta gamma delta    V0 
 TRUE  TRUE  TRUE  TRUE 


$covariateModel
$covariateModel$beta
named list()

$covariateModel$gamma
named list()

$covariateModel$delta
named list()

$covariateModel$V0
named list()

### 推定するパラメータとその初期値の設定

In [17]:
setPopulationParameterInformation(beta_pop = list(initialValue = 1e-4),
                                  gamma_pop  = list(initialValue = 3),
                                  delta_pop = list(initialValue = 1),
                                  V0_pop = list(initialValue = 1e+4),
                                  omega_beta = list(initialValue = 1),
                                  omega_gamma = list(initialValue = 1),
                                  omega_delta = list(initialValue = 1),
                                  omega_V0 = list(initialValue = 1))

In [18]:
getPopulationParameterInformation()

name,initialValue,method
<chr>,<dbl>,<chr>
beta_pop,1e-04,MLE
gamma_pop,3e+00,MLE
delta_pop,1e+00,MLE
V0_pop,1e+04,MLE
omega_V0,1e+00,MLE
omega_beta,1e+00,MLE
omega_delta,1e+00,MLE
omega_gamma,1e+00,MLE
a,1e+00,MLE


### Taskの設定

In [19]:
getScenario()

$tasks
  populationParameterEstimation conditionalDistributionSampling 
                           TRUE                            TRUE 
      conditionalModeEstimation         standardErrorEstimation 
                           TRUE                           FALSE 
        logLikelihoodEstimation                           plots 
                          FALSE                            TRUE 

$linearization
[1] FALSE

$plotList
 [1] "indfits"                  "parameterdistribution"   
 [3] "covariancemodeldiagnosis" "covariatemodeldiagnosis" 
 [5] "obspred"                  "vpc"                     
 [7] "residualsscatter"         "residualsdistribution"   
 [9] "randomeffects"            "saemresults"

In [20]:
senario = getScenario()
senario$tasks = c(populationParameterEstimation = TRUE, conditionalDistributionSampling = TRUE, conditionalModeEstimation = TRUE,
                  standardErrorEstimation = TRUE, logLikelihoodEstimation = TRUE, plots = TRUE)
setScenario(senario)

In [21]:
getScenario()

$tasks
  populationParameterEstimation conditionalDistributionSampling 
                           TRUE                            TRUE 
      conditionalModeEstimation         standardErrorEstimation 
                           TRUE                            TRUE 
        logLikelihoodEstimation                           plots 
                           TRUE                            TRUE 

$linearization
[1] FALSE

$plotList
 [1] "indfits"                  "parameterdistribution"   
 [3] "covariancemodeldiagnosis" "covariatemodeldiagnosis" 
 [5] "obspred"                  "vpc"                     
 [7] "residualsscatter"         "residualsdistribution"   
 [9] "randomeffects"            "saemresults"

### RUN

結果のフォルダが上書きされるので注意

In [22]:
startTime <- Sys.time()
runScenario()
endTime <- Sys.time()

sink("outputs/08_01/systime.txt"); print(endTime - startTime,append=T)

### プロット

In [23]:
library(ggplot2)

In [29]:
pltTemp <- plotIndividualFits(obsName = "y1",
                              setting = list(ylog = TRUE,
                                             xlim = c(0, 20),
                                             ylim = c(100, 1e+8)))
    ggsave(pltTemp, filename = "outputs/08_01/plot_indivfit.png", h = 40, w = 12)

### プロジェクトの保存

In [31]:
saveProject() # update current model